In [1]:
import platform
import datetime
import sys
import os


your_name, your_code, work_counter = "杜培博", 200507340136, 9
print(f'我是：{your_name}，学号：{your_code}，这是我的第 {work_counter} 次作业。')
print('现在是', datetime.datetime.now())
print('我的机器信息如下：')
print(platform.uname())

print('\n当前工作目录：')
print(os.getcwd())
print('\n当前目录的文件信息如下：')
print(os.listdir())
print('\n我安装包的情况:')
!pip list

我是：杜培博，学号：200507340136，这是我的第 9 次作业。
现在是 2021-11-14 20:25:45.237300
我的机器信息如下：
uname_result(system='Windows', node='LAPTOP-5PHEQ8I3', release='10', version='10.0.18362', machine='AMD64', processor='AMD64 Family 23 Model 24 Stepping 1, AuthenticAMD')

当前工作目录：
E:\jupyter\curriculum_code\da_tow\week_10

当前目录的文件信息如下：
['.ipynb_checkpoints', 'god_view.ipynb', '第十次作业.ipynb']

我安装包的情况:
Package                            Version

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.



---------------------------------- -------------------
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 1.10.0
anaconda-project                   0.8.3
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.4.2
astropy                            4.0.2
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
audioread                          2.1.9
autopep8                           1.5.4
Babel                              2.8.1
backcall                           0.2.0
backports.functools-lru-cache      1.6.1
backports.shutil-get-terminal-size 1.0.0
backports.tempfile                 1.0
backports.weakref                  1.0.post1
baidu-aip                          2.2.18.0
bcrypt                           

# 1 朴素贝叶斯实现

## 训练函数

In [2]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

## 封装BayesLaplace

In [3]:
class BayesLaplace:
    def fit(datasets):
        good_el_dic = {}  # P(*|好瓜)
        for j in datasets.columns.to_list()[0:-1]:
            for i in datasets[j].unique():
                if "是" in datasets.groupby(j)["好瓜"].value_counts()[i]:
                    good_el_dic[i] = round((datasets.groupby(j)["好瓜"].value_counts()[i]["是"] + 1) / (
                            len(datasets[datasets["好瓜"] == "是"]) + len(datasets[j].unique())), 3)  # 拉普拉斯
                else:
                    good_el_dic[i] = round(1 / (len(datasets[datasets["好瓜"] == "是"]) + len(datasets[j].unique())),
                                           3)  # 拉普拉斯
        P_good = round((len(datasets[datasets["好瓜"] == "是"]) + 1) / (len(datasets) + 2), 2)

        bad_el_dic = {}  # P(*|坏瓜)
        for j in datasets.columns.to_list()[0:-1]:
            for i in datasets[j].unique():
                if "否" in datasets.groupby(j)["好瓜"].value_counts()[i]:
                    #                 bad_el_dic[i] = round((datasets.groupby(j)["好瓜"].value_counts()[i]["否"]+1)/(datasets[j].value_counts()[i]+len(datasets[j].unique())), 2)    # 平滑
                    bad_el_dic[i] = round((datasets.groupby(j)["好瓜"].value_counts()[i]["否"] + 1) / (
                            len(datasets[datasets["好瓜"] == "否"]) + len(datasets[j].unique())), 3)
                else:
                    #                 P_bad[i] = round(1/(datasets[j].value_counts()[i]+len(datasets[j].unique())), 2)      # 平滑
                    bad_el_dic[i] = round(1 / (len(datasets[datasets["好瓜"] == "否"]) + len(datasets[j].unique())),
                                          3)  # 平滑
        P_bad = round((len(datasets[datasets["好瓜"] == "否"]) + 1) / (len(datasets) + 2), 3)
        return good_el_dic, P_good, bad_el_dic, P_bad

    def predict(datasets):
        y_pre = []
        for i in range(0, len(datasets)):
            test = datasets.iloc[i, :-1].to_list()
            P_g = P_good
            P_b = P_bad
            for i in test:
                P_g = P_g * good_el_dic[i]
            for i in test:
                P_b = P_b * bad_el_dic[i]
            if P_g > P_b:
                y_pre.append("是")
            else:
                y_pre.append("否")
        return pd.Series(y_pre)

## 导入数据

In [4]:
datasets = pd.DataFrame([["青绿", "蜷缩", "浊响", "清晰", "凹陷", "硬滑", "是"],
                         ["乌黑", "蜷缩", "沉闷", "清晰", "凹陷", "硬滑", "是"],
                         ["乌黑", "蜷缩", "浊响", "清晰", "凹陷", "硬滑", "是"],
                         ["青绿", "蜷缩", "沉闷", "清晰", "凹陷", "硬滑", "是"],
                         ["浅白", "蜷缩", "浊响", "清晰", "凹陷", "硬滑", "是"],
                         ["青绿", "稍蜷", "浊响", "清晰", "稍凹", "软粘", "是"],
                         ["乌黑", "稍蜷", "浊响", "稍糊", "稍凹", "软粘", "是"],
                         ["乌黑", "稍蜷", "浊响", "清晰", "稍凹", "硬滑", "是"],
                         ["乌黑", "稍蜷", "沉闷", "稍糊", "稍凹", "硬滑", "否"],
                         ["青绿", "硬挺", "清脆", "清晰", "平坦", "软粘", "否"],
                         ["浅白", "硬挺", "清脆", "模糊", "平坦", "硬滑", "否"],
                         ["浅白", "蜷缩", "浊响", "模糊", "平坦", "软粘", "否"],
                         ["青绿", "稍蜷", "浊响", "稍糊", "凹陷", "硬滑", "否"],
                         ["浅白", "稍蜷", "沉闷", "稍糊", "凹陷", "硬滑", "否"],
                         ["乌黑", "稍蜷", "浊响", "清晰", "稍凹", "软粘", "否"],
                         ["浅白", "蜷缩", "浊响", "模糊", "平坦", "硬滑", "否"],
                         ["青绿", "蜷缩", "沉闷", "稍糊", "稍凹", "硬滑", "否"]],
                        columns=["色泽", "根蒂", "敲声", "纹理", "脐部", "触感", "好瓜"])

## 实例化一个估计器，训练模型

In [5]:
a = BayesLaplace
good_el_dic, P_good, bad_el_dic, P_bad = a.fit(datasets)

## 模型预测

In [6]:
y_pre = a.predict(datasets)
print("预测值时是：\n", y_pre)

预测值时是：
 0     是
1     是
2     是
3     是
4     是
5     是
6     否
7     是
8     否
9     否
10    否
11    否
12    是
13    否
14    是
15    否
16    否
dtype: object


## 准确率

In [7]:
y_test = datasets["好瓜"]
acc = round(len(y_pre[y_test==pd.Series(y_pre)])/len(datasets), 3)
print("准确率为：", acc)

准确率为： 0.824


## 查准率

In [8]:
lb = LabelEncoder()
y_test = lb.fit_transform(y_test)
y_pre = lb.fit_transform(y_pre)
temp = 0
for i, j in zip(y_test, y_pre):
    if (i == j) & (i == 1):
        temp += 1
precision = temp / len(y_pre[y_pre == 1])
print("查准率：", precision)

查准率： 0.7777777777777778


## 查全率

In [9]:
recall = temp/len(y_test[y_test == 1])
print("查全率", recall)

查全率 0.875


## F1_score

In [10]:
f1 = 2*recall*precision/(recall+precision)
print("f1", f1_score(y_test, y_pre))

f1 0.823529411764706


# 朴素贝叶斯掉包

In [11]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB

iris = datasets.load_iris()
train = iris.data[:, 1:3]
test = iris.target

x_train, x_test, y_train, y_test = train_test_split(train, test, test_size=0.2, random_state=1113)

estimator = MultinomialNB()
estimator.fit(x_train, y_train)
y_p = estimator.predict(x_test)
print("预测结果：\n", y_p == y_test)

预测结果：
 [ True False False  True  True False  True False False False  True  True
  True False  True False  True False False  True  True False False  True
  True False  True  True  True  True]


# 机器学习三要素

# Y = F(X)

## 数据：特征值，就是X

## 模型：函数，就是F()

## 标签：目标值,就是Y

# 4 生成式模型和判别式模型

## 生成式模型：生成式模型学习了每个类别的边界，它包含了更多信息，可以用来生成样本,生成式模型是有多少个特征就建多少个模，再去计算P(y|x) = P(x,y)/P(x)

## 判别式模型：判别式模型是在寻找一个决策边界，通过该边界来将样本划分到对应类别， 判别式模型是对条件概率建模，直接计算P(y|x)

# 5 过拟合：过拟合就是学习到的特征太多或学到了一些没用的特征，例如：对树叶的预测，当数据量较少时，学到了“树叶边缘是锋利的”这个特征，当去预测一个边缘不锋利的树叶的时候，就会判断出它不是树叶